In [11]:
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt

# import matplotlib
# import sklearn
import torch

from torch import nn, optim

from IPython.display import display
plt.style.use("ggplot")

print("package版本信息：")
print("numpy:      ", np.__version__)
# print("pandas:     ", pd.__version__)
# print("matplotlib: ", matplotlib.__version__)
# print("sklearn:    ", sklearn.__version__)
print("PyTorch:     ", torch.__version__)

package版本信息：
numpy:       1.21.5
PyTorch:      1.2.0


# Pytorch-Transformer

参考文档:
+ [Pytorch tutorial -> Transformer tutorial](https://pytorch.org/tutorials/beginner/transformer_tutorial.html)
+ [Pytorch -> Transformer Layers](https://pytorch.org/docs/stable/nn.html#transformer-layers)

这里介绍了Pytorch里 transformer 层的实现模块.

主要有如下几个类：
+ `nn.Transformer`：一步到位构建整个Transformer模型，底层使用的是下面的API
+ `nn.TransformerEncoder`：Encoder层，包含了多个 `TransoformerEncoderLayer`
+ `nn.TransformerDecoder`：Decoder层，包含了多个 `TransformerDecoderLayer`
+ `nn.TransformerEncoderLayer`：单层Encoder
+ `nn.TransformerDecoderLayer`：单层Decoder

但是需要注意的是，上述各种层的**底部都是调用的`activation.py`中的`MultiheadAttention`类**。

## Multi-Head Attention Layer

[官方文档](https://pytorch.org/docs/stable/generated/torch.nn.MultiheadAttention.html#torch.nn.MultiheadAttention).

它是`TransformerEncoderLayer`和`TransformerDecoderLayer`层中 **self-attention 层**的实现。  
需要注意的是，pytorch源码中 `MultiheadAttention` 被放在了 Non-linear Activations 这一类中（并且是放在`activation.py`文件中的）。


### 实例化参数
`MultiheadAttention(embed_dim, num_heads, dropout=0.0, bias=True, add_bias_kv=False, add_zero_attn=False, kdim=None, vdim=None)`  
+ `embed_dim`，整个MHA的输入/输出特征维度。  
  **原始论文里，MHA的输入/输出特征维度可以是不一样的，但是在Pytorch的实现里，强制要求它们是一样的**（这个限制未来版本可能会取消）。
+ `num_heads`，head数量，注意，分配到每个head的维度 = embed_dim/num_heads.
+ `kdim`，自定义 key 中的特征维度，它是下面`forward()`方法里`key`对应的特征维度，默认下=`embed_dim`
+ `vidm`，自定义 value 中的特征维度，`forward()`方法里`value`对应的特征维度，默认=`embed_dim`
+ `bias`，
+ `add_bias_kv`
+ `add_zero_attn`
+ `batch_first`，早期版本的pytorch中，会要求下面`forward()`方法里输入数据的各个维度顺序必须为：`(序列长度，batch_size, embedding_dimension)`，后来提供了这个`batch_first`参数，用于适应一下输入数据的维度变化.


### 前向传播

`MultiheadAttention.forward(query, key, value, key_padding_mask=None, need_weights=True, attn_mask=None)`
+ `query`, `key`, `value`，MHA的三个输入矩阵——这三个输入向量会被用于生成对应的Q,K,V
  + `query.shape` = $(L, N, E_q)$，`key.shape` = $(S, N, E_k)$，`value.shape` = $(S, N, E_v)$.   
    + $N$ 是 batch size
    + $E$ 是 embedding 维度，$E_q$=`embed_dim`, $E_k$=`kdim`, $E_v$=`vdim`
    + $L$ 是**target**序列的长度——因为它决定了MHA输出的序列长度
    + $S$ 是**source**序列的长度
  + Transformer中MHA会**在3个地方使用，这3个地方会影响 query, key, value 的内容**
    + Encoder中 query, key, value 都是来自于样本的**输入序列**，此时 $L=S$，`kdim == vdim`
    + Decoder的 **Masked MHA** 中 query, key, value 都来自于样本的**输出序列**，此时 $L=S$, `kdim == vdim`
    + Decoder的 **decoder-encoder attention**中，query 来自于样本的**输出序列**，key, value 都是**Encoder最后一层的输出**，此时可能 $L\neq S$, `kdim != vdim`
  + **有关序列维度的说明**
    + 输入的 `query`, `key`, `value` 不是self-attention的输入，它们需要经过各自的投影矩阵 $W_Q \in (E_q, E_q)$, $W_K \in (E_d, E_k)$, $W_V \in (E_d, E_v)$ 的转换，上面这3个参数矩阵的维度，就是MHA里初始化的维度
    + 线性投影过程如下（这里把batch的维度提前了）：   
    $q = query * W_Q^T \in (N, L, E_d)$,   
    $k = key * W_K^T \in (N, S, E_d)$,   
    $v = value * W_V^T \in (N, S, E_d)$  
    + 可以看出，参数的 `kdim` 和 `vdim` 只是用于设置投影矩阵 $W_K, W_V$ 的维度，这两个矩阵的另一个维度被设置成了 $E_q$=`embed_dim`，经过上述三个投影矩阵的变换之后，`query`, `key`, `value` 的维度都变成了 $E_q$，也就是 `embed_dim`
    + 后续进行的MHA操作为：
    $$softmax(\frac{q*k^T}{\sqrt{E_d}}) * v$$
    $$((N, L, E_d) * (N, E_d, S)) * (N, S, E_d) \rightarrow (N, L, E_d)$$
    调用的是`torch.bmm()`算子，会在每个批次上做上述操作。
    + 可以看出，最终决定输出序列长度的是 `query` 的序列长度 $L$，而 `key` 和 `value` 的序列长度必须一致，它们会在中间计算过程中相互抵消掉。


+ `key_padding_mask`：填充位置的掩码，shape=$(N, S)$   
  + 对于一个batch的训练数据来说，batch size 为 $N$ 表示有 $N$ 个序列，但是**每个序列的长度（也就是单词个数）是不一样的**，`key`,`value`中的 $S$ 都是指最长序列的长度，其他的序列就需要对缺少的单词位置做填充(padding)，padding位置的词通常使用全为0的embeding 来表示。   
  + 为了不让MHA对这些位置的词产生注意力，需要记录这些padding位置，每个batch都使用这样一个 $(N, S)$ 的矩阵来标识padding的位置，该矩阵的每一行对应batch中的一个序列，值为`[False, False, True, True]` 或者 `[0, 0, 1, 1]` 的形式，其中结尾`True`或者1标识的就是padding的位置。   
  + 需要注意的是，这里padding矩阵实际生效位置是在`query`和`key`做完矩阵乘法得到shape=$(N,L,S)$的中间矩阵 P 之后才使用的，由于只有`key`使用了padding掩码，只需要在 $N$ 个序列中对 $S$ 这一个维度标识padding位置即可，所以使用的padding矩阵shape=$(N,S)$. pytorch中的实现就是对中间矩阵 P 的 $S$ 维度，按照 `[False,False,True,True]` 中的标记，对应`True`的位置赋值为`-inf`，从而在后续的softmax中，该位置得到的概率为0.


+ `attn_mask`，**只在Decoder中使用**，主要是解码时不让当前位置的词和后面要预测的词序列进行Attention.   
  + shape=$(L,S)$ 或者 $(N * num\_heads,L,S)$，如果是 2D 的矩阵，那么batch中的每个序列都会使用这个，如果是 3D 的矩阵，则会指定每个batch 的掩码
  + 它的生效位置也是在`query`和`key`相乘得到shape=$(N,L,S)$的中间矩阵 P 之后使用，此时batch中**每个**序列的输入`query`部分（shape=$(L,E)$）和`key`部分（shape=$(S,E)$）相乘得到了一个 $(L,S)$ 的矩阵 P，矩阵中列表示`query`里的每个词，行表示`key`中的每个词，如果不想让`query`中第 $i$ 个词看到`key`中第 $j$ 个词之后的内容，只需要让 `P[i, j:]` 这部分为 `-inf` 即可，因此这个掩码的shape=$(N,L,S)$.
  + 如果是在Decoder中的**Masked MHA**这里使用，此时输入的`query`,`key`,`value`都是来源于同一个地方，`query`和`key`维度相等，此时 P 是一个方阵，并且右上三角均为`-inf` —— 这是好理解的部分，也是正常它要解决的问题部分。
  + 如果是在Decoder中的**encoder-decoder attention**层使用，此时输入的`query`是来自于之前的Masked MHA层，属于输出序列的内容，但是`key`,`value`是来自于Encoder，属于输入序列的内容，此时才会出现`query`和`key`长度不等的情况——这种情况下虽然也可以使用掩码，但是它的含义就不那么好解释了。


+ `need_weights`，bool，表示是否需要返回`attn_output_weigths`，默认为`True`


+ `forward()`方法返回值
  + `attn_output`，shape= $(L, N, E_q)$，$L$ 由 输入的`query`序列长度决定，$E$=`embed_dim`.
  + `attn_output_weights`，shape=$(N, L, S)$，这个权重应该是每个输入序列中，每个word的attention权重.
  

### 一些说明

Pytorch中的Multi-Head Attention的实现和论文《All you need is transformer》中有一些不一样的地方，总结如下：

+ pytorch在实现MHA时，强制要求了输入的特征维度和输出的特征维度必须一致，也就是实例化时的`embed_dim`参数，但是在原论文中没有这个限制。   
原论文中，输入的特征维度是`query`里的embed_dim，输出的特征维度由`value`的embed_dim决定。

+ pytorch中MHA实例化参数 `kdim`,`vdim` 和含义和原论文里的 $d_k$, $d_v$ 不一样，如下所示.   
原论文中的 $d_k$, $d_v$ 指的是 query, key 经过参数矩阵 $W$ 变换之后的 embed_dim，而pytoch中的 `kdim`,`vdim` 指定的是输入的 `key`,`value` 中，没有经过 $W$ 变换之前的特征维度，$W$ 这个参数矩阵设置的时候，其中一个维度就是 `kdim` （或者 `vdim`），另一个维度就是 `embed_dim`，因此 `key`, `value` 经过 $W$ 变换之后，维度就变成了 `embed_dim` ——这是 pytorch强制规定的。

<img src="images/multi-head-attention-formula.png" width="60%" align="left">

### 使用示例

In [2]:
embed_dim, num_heads = 512, 8
batch_size, target_len, source_len = 5, 10, 20

query = torch.rand(target_len, batch_size, embed_dim)
key = torch.rand(source_len, batch_size, embed_dim)
value = torch.rand(source_len, batch_size, embed_dim)

In [3]:
multihead_attn = nn.MultiheadAttention(embed_dim, num_heads)
attn_output, attn_output_weights = multihead_attn(query, key, value)
print(attn_output.shape)
print(attn_output_weights.shape)

torch.Size([10, 5, 512])
torch.Size([5, 10, 20])


In [4]:
kdim, vdim = 256, 256
key = torch.rand(source_len, batch_size, kdim)
value = torch.rand(source_len, batch_size, vdim)

multihead_attn = nn.MultiheadAttention(embed_dim, num_heads, kdim=kdim, vdim=vdim)
attn_output, attn_output_weights = multihead_attn(query, key, value)

print(attn_output.shape)
print(attn_output_weights.shape)

torch.Size([10, 5, 512])
torch.Size([5, 10, 20])


## Encoder Layer

### 单层Encoder

[官方文档](https://pytorch.org/docs/stable/generated/torch.nn.TransformerEncoderLayer.html#torch.nn.TransformerEncoderLayer)，它内部包含了一个 self-attention 层 + 一个 feedforward 层.

`nn.TransformerEncoderLayer(d_model, nhead, dim_feedforward=2048, dropout=0.1, activation='relu')`  

+ 实例化参数
  + `d_model`，输入序列中，每个word的特征个数——它同时也决定了输出序列里每个word的特征个数
  + `nhead`，multiheadattention中的head个数
  + `dim_feedforward`，这个维度设置的是前馈神经网络的节点个数，前馈神经网络的输出节点数还是 `d_model`
  + `dropout`，dropout比例，默认 0.1  


+ 前向传播`forward(src, src_mask=None, src_key_padding_mask=None)`
  + `src`，输入的sequence, `shape` = $(S, N, E)$
  + `src_mask`，输入sequence的mask
  + `src_key_padding_mask`，每个batch中src的padding矩阵  


+ `forward()`方法返回值  
  它返回的是和 `src` shape 相同的 tensor.


Encoder的内部只有**一层**`MultiheadAttention`：  
`Encoder.forward()`实际执行时，调用方式为`MultiheadAttention.forward(src, src, src)`——`src`会同时作为 query, key, value 传入 self-attention。



### 多层Encoder

`TransformerEncoder(encoder_layer, num_layers, norm=None)`
+ 参数：
  + `encoder_layer`，是一个`TransformerEncoderLayer`类的实例对象
  + `num_layers`，指定堆叠的 Encoder 层数
+ `forward()`方法和`TransformerEncoderLayer`的一致

In [5]:
d_model, num_heads, dim_ffn = 512, 8, 2048
batch_size, source_len = 10, 32
src = torch.rand(source_len, batch_size, d_model)

encoder_layer = nn.TransformerEncoderLayer(d_model=512, nhead=8, dim_feedforward=dim_ffn)
out = encoder_layer(src)

print("src.shape: ", src.shape)
print("out.shape: ", out.shape)

src.shape:  torch.Size([32, 10, 512])
out.shape:  torch.Size([32, 10, 512])


In [8]:
multi_encoder_layer = nn.TransformerEncoder(encoder_layer=encoder_layer, num_layers=3)
out = multi_encoder_layer(src)

print("src.shape: ", src.shape)
print("out.shape: ", out.shape)

src.shape:  torch.Size([32, 10, 512])
out.shape:  torch.Size([32, 10, 512])


## Decoder Layer

### 单层Decoder

[官方文档](https://pytorch.org/docs/stable/generated/torch.nn.TransformerDecoderLayer.html#torch.nn.TransformerDecoderLayer)

`TransformerDecoderLayer(d_model, nhead, dim_feedforward=2048, dropout=0.1, activation='relu')`   

+ 实例化参数：和 Encoder 一致
  + `d_model`，输入序列中，每个word的特征个数——它同时也决定了输出序列里每个word的特征个数
  + `nhead`，multiheadattention中的head个数
  + `dim_feedforward`，这个维度设置的是前馈神经网络的节点个数，前馈神经网络的输出节点数还是 `d_model`
  + `dropout`，dropout比例，默认 0.1


+ 前向传播`forward(tgt, memory, tgt_mask=None, memory_mask=None, tgt_key_padding_mask=None, memory_key_padding_mask=None)`  
  这个方法和 Encoder 不一样，它多了一个 memory. 
  + `tgt`，输入的序列，`shape`= $(S, N, E)$，
  + `memory`，来自 encoder 层（通常是最后一层）的输出, `shape`=$(S, N, E)$.
  + `tgt_mask`，
  + `memory_mask`
  + `tgt_key_padding_mask`，
  + `memory_key_padding_mask`

> `Decoder`内部会调用两层`MultiheadAttention`
> 1. 第一层调用时为`MultiheadAttention.forward(tgt, tgt, tgt)`
> 2. 第二层调用时为`MultiheadAttention.forward(tgt, memory, memory)`


+ `forward()`方法返回值  
它返回的是和 `tgt` shape 相同的 tensor，


### 多层Decoder

`TransformerDecoder(decoder_layer, num_layers, norm=None)`   
参数同 `TransformerEncoder` 类.

In [7]:
d_model, nhead, dim_ffn = 512, 8, 2048
batch_size, source_len = 10, 32
tgt = torch.rand(source_len, batch_size, d_model)
memory = torch.rand(source_len, batch_size, d_model)

decoder_layer = nn.TransformerDecoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=dim_ffn)
decoder_out = decoder_layer(tgt, memory)

print("tgt.shape: ", tgt.shape)
print("memory.shape: ", memory.shape)
print("decoder_out.shape: ", decoder_out.shape)

tgt.shape:  torch.Size([32, 10, 512])
memory.shape:  torch.Size([32, 10, 512])
decoder_out.shape:  torch.Size([32, 10, 512])


In [9]:
multi_decoder_layer = nn.TransformerDecoder(decoder_layer=decoder_layer, num_layers=3)
decoder_out = multi_decoder_layer(tgt, memory)

print("tgt.shape: ", tgt.shape)
print("memory.shape: ", memory.shape)
print("decoder_out.shape: ", decoder_out.shape)

tgt.shape:  torch.Size([32, 10, 512])
memory.shape:  torch.Size([32, 10, 512])
decoder_out.shape:  torch.Size([32, 10, 512])


## Transformer类

[官方文档](https://pytorch.org/docs/stable/generated/torch.nn.Transformer.html#torch.nn.Transformer)

`nn.Transformer(d_model=512, nhead=8, num_encoder_layers=6, num_decoder_layers=6, dim_feedforward=2048, dropout=0.1, activation=<function relu>, custom_encoder=None, custom_decoder=None, layer_norm_eps=1e-05, batch_first=False, norm_first=False, device=None, dtype=None)`

+ 实例化参数
  + `d_model`，输入/输出的embedding dim，默认512
  + `nhead`
  + `num_encoder_layers`
  + `num_decoder_layers`
  + `dim_feedforward`
  + `dropout`
  + `activation`
  
+ 正向传播`forward(src, tgt, src_mask=None, tgt_mask=None, memory_mask=None, src_key_padding_mask=None, tgt_key_padding_mask=None, memory_key_padding_mask=None)`
  + `src`，shape=$(S,N,E)$
  + `tgt`，shape=$(T,N,E)$
  + `src_mask`，shape=$(S,S)$
  + `tgt_mask`，shape=$(T,T)$
  + `memory_mask`，shape=$(T,S)$
  + `src_key_padding_mask`，shape=$(N,S)$
  + `tgt_key_padding_mask`，shape=$(N,T)$
  + `memory_key_padding_mask`，shape=$(N,S)$

----------------------------

# Huggingface-Transformer

参考：
+ 官方网站 [huggingface-transformer](https://huggingface.co/docs/transformers/index)
+ github地址 [huggingface/transformers](https://github.com/huggingface/transformers)

这里介绍huggingface编写的transformer包，它里面基于transformer结构，实现了各种模型，包括BERT, DistilBERT, GPT2 等等.


## 基本框架

有关transformer包的设计理念可以参考官方文档 [Philosophy](https://huggingface.co/docs/transformers/philosophy).

transformer 包为了保持简洁，尽量减少了抽象接口，每个模型只有3个必要的抽象部分，这里以NLP处理为例：

1. [Configuration](https://huggingface.co/transformers/main_classes/configuration.html)  
基础的类是 `PretrainedConfig`（好像也只有这一个类），用于封装各个模型所需要的参数配置，同时也可以方便的 **导出/导入** 参数配置。  
不同的模型有自己的配置类，但是都是继承于此类。

2. [Models](https://huggingface.co/transformers/main_classes/model.html)  
基础的类有如下三个，用于构建模型：
   + `PreTrainedModel`，pytorch实现的模型都继承了这个类，继承于 `torch.nn.Module` 类.
   + `TFPreTrainedModel`，Tensorflow2.0实现的模型都继承了这个类，继承于 `tf.keras.Model` 类.
   + `FlaxPreTrainedModel`.  


3. 预处理类，对于NLP任务来说是 [Tokenizer](https://huggingface.co/transformers/main_classes/tokenizer.html)  
包含两个基础的类，用于将文本特征转换成transformer能处理的序列特征：
   + `PreTrainedTokenizer`，这个是纯Python实现，速度比较慢
   + `PreTrainedTokenizerFast`，这个是基于Rust实现，速度比较快   
   
   对于视频或音频任务来说，是feature extrator类。

上面这三个基础模块，都有如下两个方法用于**导入/导出**对应的配置：
+ `.from_pretrained()`，导入预训练模型的 config/model/tokenizer
  + `pretrained_model_name_or_path`，指定预训练模型的名称或者本地路径
  + `cache_dir`，指定模型的缓存位置
  + `force_download`，bool，指定是否强制下载模型
  + `local_files_only`，bool，指定是否只使用本地模型
  + `mirror`，指定下载的镜像地址


+ `.save_pretrained()`，导出训练好模型的 config/model/tokenizer

**第一次使用这些语句的时候，如果本地没有对应的模型，会下载这些模型**。
  
在上面的3个基础子模块之上，还提供了如下几个子模块工具：

+ [Pipepline](https://huggingface.co/transformers/main_classes/pipelines.html)  
用于快速使用模型，是对上面三个类的封装
+ [Trainer](https://huggingface.co/transformers/main_classes/trainer.html)  
用于快速训练或者 fine-tune 模型


transformer源码中，所有的模型结构都存放在 `src/transformers/models`文件夹下，每个模型对应于一个文件夹（比如BERT对应的就是`bert`文件夹），每个模型的文件夹内，主要有如下几个`.py`文件，其中的 `xxx` 是对应模型的名称：
+ **`configuration_xxx.py`——KEY**，编写了该模型对应的`PretrainedConfig`子类，比如BERT就是`BertConfig`。
+ **`modeling_xxx.py`——KEY**，存放了pytorch实现的模型结构，要用到的模型类都放在这里面
+ `modeling_tf_xxx.py`，存放tensorflow实现的模型结构
+ `modeling_flax_xxx.py`
+ **`tokenization_xxx.py`——KEY**，用于 分词的 实现类，比如BERT就是`BertTokenizer`类
+ `tokenization_xxx_fast.py`，快速分词的实现类
+ `convert_*.py`，用于将其它配置文件转换成对应的模型。

## Configuration

----------------

## Tokenizer

官方文档 [Tokenizer](https://huggingface.co/docs/transformers/v4.17.0/en/main_classes/tokenizer)

`Tokenizer`类是处理transformer结构输入的主要类，它用于将文本类的数据转换成transformer结构能够接受的序列数据.  

所有的模型里，Tokenizer 都有两种实现：
  1. 基于python代码的实现，对应于`PreTrainedTokenizer`类
  2. 基于Rust Library的**快速**实现，对应于`PreTrainedTokenizerFast`类.


上述两个类又都是基于`PreTrainedTokenizerBase`类，该类实现了分词处理的主要方法，具体功能如下所示：
  + 对文本进行分词，并将分词后的word映射到对应词典的id
  + 生成特殊的Tokens，比如mask等。
  
  
有关 tokenizer 的使用，可以参考如下官方教程：
+ [Preprocessing -> NLP](https://huggingface.co/docs/transformers/preprocessing#nlp)
+ [Glossary -> Model inputs](https://huggingface.co/docs/transformers/glossary#model-inputs)

### 实例化Tokenizer

`PreTrainedTokenizer`类虽然有实例化参数，但是通常会**使用`.from_pretrained()`方法从文件中读取配置，生成对应的Tokenizer对象**。  

通常该对象会有如下属性：
+ `name_or_path`：读取的配置路径
+ `vocab_file_names`：字典文件名
+ `vocab_size`：单词字典的大小
+ `vocab`：单词字典，是一个OrderedDict
+ `ids_to_tokens`：id和token的映射字典，也是一个OrderedDict
+ `wordpiece_tokenizer`：底层使用的分词器
+ `max_model_input_sizes`
+ `max_len_single_sentence`
+ `model_max_length`

此外，还有一些特殊token的字符和id，比如：
+ `pad_token`, `pad_token_id`
+ `sep_token`, `sep_token_id`
+ `cls_token`, `cls_token_id`



### 使用Tokenizer

`PreTrainedTokenizer`类有如下常用的方法。



#### `__call__()`

参数如下：
+ `text`，输入的句子或者句子序列，可接受的输入类型有：
  + `str`，单个句子
  + `List[str]`，多个句子。注意，如果这里是 list of word 的形式，那么需要设置`is_split_into_words=True`。
  + `List[List[str]]`，一个batch的句子
+ `text_pair`，输入的文本对，可接受的输入类型同`text`
+ `max_length`：设置可接受的最大句子长度
+ `padding`，补全策略，有如下几种输入：
  + `True` or `longest`，使用padding，此时按照最长的句子进行padding
  + `max_length`，按照初始化指定的`max_length`参数或者模型的`model_max_length`填充
  + `False` 或者 `do_not_pad`，不做padding，这是**默认值**。此时返回的数据中，一个batch中的各个句子会有不同的长度。
+ `truncation`，指定是否截断
+ `stride`
+ `return_tensors`：返回值的类型
  + `tf`：返回`tf.constant`对象
  + `pt`：返回`torch.Tensor`对象
  + `np`：返回`numpy.ndarray`对象


`__call__()`方法返回的是一个`BatchEncoding`类对象，封装了返回的结果，它类似于一个dict，有如下重要的keys:
  + `input_ids`，分词后每个 token 对应 vocabulary 的 index —— 这个被**作为输入模型的数据**.
  + `attention_mask`，表明填充的 token 位置，**填充位的 token 对应于 0**.  
  因为一个batch中不同的句子长度不一样，短一些的句子会被padding成同样的长度，此时这些padding的位置需要被记录下来，用于告知模型这部分只是填充使用，不需要参与self-attention。
  + `token_type_ids`：通常用于问答对的句子中，它也是一个{0,1}掩码的形式，0表示属于问题的句子，1表示属于答案的句子

#### `tokenize()`
将文本转换成分词后的 tokens，返回的是 `List[str]`，表示的是文本对应 **分词后** 的 tokens .

  
#### `encode()`
将文本转换成 sequences of ids.
+ 它接受的参数和 `__call__()` 方法一样
+ 返回的是 `List[int]`，表示的是文本分词后 tokens 对应的 tokenized ids.


#### `decode()`
用于将分词后的 tokens id 转成分词后的文本


#### 其他方法

+ `convert_ids_to_tokens()`
+ `conver_tokens_to_ids()`
+ `conver_tokens_to_string()`

### 使用示例

In [2]:
cd ..

D:\Project-Workspace\Python-Projects\DataAnalysis


In [3]:
pwd

'D:\\Project-Workspace\\Python-Projects\\DataAnalysis'

In [4]:
# 导入 BERT tokenizer 的设置
from transformers import BertTokenizer
model_path = r"bert-pretrained-models\bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(pretrained_model_name_or_path=model_path, local_files_only=True)

D:\Miniconda3\envs\deep-learning\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]


In [5]:
# 查看分词表大小
print("tokenizer.vocab_size: ", tokenizer.vocab_size)
# 查看分词器所使用的的词表， 这个打印会很长
# tokenizer.vocab

tokenizer.vocab_size:  30522


In [6]:
tokenizer.basic_tokenizer

In [7]:
print(tokenizer.all_special_tokens, ':', tokenizer.all_special_ids)
print(tokenizer.cls_token, ': ',tokenizer.cls_token_id)
print(tokenizer.mask_token, ': ', tokenizer.mask_token_id)
print(tokenizer.pad_token, ': ', tokenizer.pad_token_id)

['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]'] : [100, 102, 0, 101, 103]
[CLS] :  101
[MASK] :  103
[PAD] :  0


In [8]:
sentence = "A Titan RTX has 24GB of VRAM"

# 查看分词后的 tokens
tokenized_words = tokenizer.tokenize(sentence)
print("tokenized_words: ", tokenized_words)

# 查看分词后的 tokens id
tokenized_ids = tokenizer.encode(sentence)
print("tokenized_ids: ", tokenized_ids)

# 从 token id 映射回 token
decoded_tokens = tokenizer.decode(tokenized_ids)
print("decoded_tokens: ", decoded_tokens)

tokenized_words:  ['a', 'titan', 'rt', '##x', 'has', '24', '##gb', 'of', 'vr', '##am']
tokenized_ids:  [101, 1037, 16537, 19387, 2595, 2038, 2484, 18259, 1997, 27830, 3286, 102]
decoded_tokens:  [CLS] a titan rtx has 24gb of vram [SEP]


In [9]:
# 直接调用 __call__() 方法，可以得到分词后的所有信息，一步到位
batch_encoding = tokenizer(sentence)
print("input_ids: ", batch_encoding['input_ids'])
print("attention_mask: ", batch_encoding['attention_mask'])
print("token_type_ids: ", batch_encoding['token_type_ids'])

input_ids:  [101, 1037, 16537, 19387, 2595, 2038, 2484, 18259, 1997, 27830, 3286, 102]
attention_mask:  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
token_type_ids:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [10]:
# 多个长度不一样的句子要使用 padding，并且由 attention_mask 区分
s1 = "this is a short sentence"
s2 = "This is a rather longer sequence. It is at least longer than the sequence 1"

batch_encoding = tokenizer([s1, s2], padding=True)

print("s1.length: ", len(batch_encoding['input_ids'][0]), "; s2.length: ", len(batch_encoding['input_ids'][1]))
print("s1['input_ids']: ", batch_encoding['input_ids'][0])
print("s2['input_ids']: ", batch_encoding['input_ids'][1])
print("s1['attention_mask']: ", batch_encoding['attention_mask'][0])
print("s2['attention_mask']: ", batch_encoding['attention_mask'][1])

s1.length:  18 ; s2.length:  18
s1['input_ids']:  [101, 2023, 2003, 1037, 2460, 6251, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
s2['input_ids']:  [101, 2023, 2003, 1037, 2738, 2936, 5537, 1012, 2009, 2003, 2012, 2560, 2936, 2084, 1996, 5537, 1015, 102]
s1['attention_mask']:  [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
s2['attention_mask']:  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


-----------------

## Model

transformer的Model基类`PreTrainedModel`有如下的一些属性或方法：
+ `.base_model`，返回当前使用的模型，是`torch.nn.Module`类对象
+ `get_input_embeddings()`，返回Embedding层，也是一个`nn.Module`类对象
+ `get_output_embeddings()`，返回输出层的 embeddings。


+ 可用的模型列表见官网 [Pretrained models](https://huggingface.co/transformers/pretrained_models.html).
  + BERT模型（名称）有：
    + bert-base-uncased
    + bert-base-cased
    + bert-large-uncased
    + bert-large-cased
    + bert-base-chinese  
    
  + Distil-Bert模型（名称）有：
    + distilbert-base-uncased
    + distilbert-base-cased
  
  这些模型都比较大，可以提前下载下来。

我一般常用的两个模型是 **BERT** 和 **Distil-BERT**，所以下面会着重关注这两个模型里的实现。

### BERT 模型
[BERT Models](https://huggingface.co/transformers/model_doc/bert.html) 包含如下类（以pytorch为例）
+ `BertConfig`，配置类.  
位于`configuration_bert.py`文件中，该文件中也只有这一个配置类
+ `BertTokenizer`，实现 `WordPiece` 分词的类.  
位于`tokenization_bert.py`文件中，该文件中还有 `BasicTokenizer` 和 `WordpieceTokenizer` 两个类，不过`BertTokenizer`会调用它们.
+ `BertForPreTrainingOutput`，记录BERT模型的输出.
+ `BertModel`，最基本的 BERT 模型类，它返回的是 BERT 模型的原始结果，包括隐藏层状态.
+ `BertForPreTraining`，在 `BertModel`的输出上加了一层处理，以下的几个模型都是如此.
+ `BertForMaskedLM`
+ `BertForNextSentencePrediction`
+ `BertForSequenceClassification`
+ 还有其他的一些基础工具类和其他任务对于的BERT模型类，这里就不介绍了。   

注意：
> huaggingface-transformer 里，**并没有使用pytorch内部提供的nn.Transformer相关的模型，而是自己重新写了一套实现逻辑**.

#### BertModel

最基本的BERT架构，输出的是最原始的BERT encoder 状态，没有添加任何多余的层.

查看源码时，会发现它的初始化方法里，有3个层：
1. `self.embeddings = BertEmbeddings(config)`  
这一层用于将输入的input_ids转成对应的词向量，底层调用的是`nn.Embedding`
2. `self.encoder = BertEncoder(config)`  
这一层内部是`self.layer = nn.ModuleList([BertLayer(config) for _ in range(config.num_hidden_layers)])`，其中的`BertLayer`
3. `self.pooler = BertPooler(config) if add_pooling_layer else None`


+ `forward(input_ids=None, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None, inputs_embeds=None, encoder_hidden_states=None, encoder_attention_mask=None, past_key_values=None, use_cache=None, output_attentions=None, output_hidden_states=None, return_dict=None)`方法

+ `forward()`方法返回的是`BaseModelOutputWithPoolingAndCrossAttentions`类对象（在`return_dict=True`时），它有如下属性：
  + `last_hidden_state`，shape=`(batch_size, sequence_length, hidden_size)`，最后一层的隐状态
  + `pooler_output`，shape=`(batch_size, hidden_size)`，最后一层中 `[CLS]` token 对应的输出
  + `hidden_states`，只有当`output_hidden_states=True`时才会返回.  
  返回的是一个 长度=13 的tuple, 每个元素对应于一层的隐状态（包括了 Embedding 层），每层的隐状态 shape=`(batch_size, sequence_length, hidden_size)`
  + `attentions`，只有当`output_attentions=True`时才会返回.  
  返回一个 长度=12 的tuple，对应于 12 层的attention，每一层的attention.shape=`(batch_size, num_heads, sequence_length, sequence_length)`，
  + `cross_attentions`，只有当`output_attentions=True`时才会返回.  
  返回一个 长度=12 的tuple，对应于 12 层的attention，每一层的attention.shape=`(batch_size, num_heads, sequence_length, sequence_length)`，
  
如果`return_dict=False`，那么返回的就是一个 tuple，其中包含的具体元素要看配置.

In [6]:
from transformers import BertConfig, BertTokenizer, BertModel

In [7]:
pwd

'D:\\Projects\\DataAnalysis'

In [43]:
# model_name = "./BERT/bert-pre-trained-models/bert-base-uncased"
model_name = "./BERT/bert-pre-trained-models/distilbert-base-uncased/"

config = BertConfig.from_pretrained(model_name, local_files_only=True)
tokenizer = BertTokenizer.from_pretrained(pretrained_model_name_or_path=model_name, local_files_only=True)
model = BertModel.from_pretrained(model_name, local_files_only=True)

print("config.__class__:    ", config.__class__)
print("tokenizer.__class__: ", tokenizer.__class__)
print("model.__class__:     ", model.__class__)

Some weights of the model checkpoint at ./BERT/bert-pre-trained-models/distilbert-base-uncased/ were not used when initializing BertModel: ['distilbert.embeddings.word_embeddings.weight', 'distilbert.embeddings.position_embeddings.weight', 'distilbert.embeddings.LayerNorm.weight', 'distilbert.embeddings.LayerNorm.bias', 'distilbert.transformer.layer.0.attention.q_lin.weight', 'distilbert.transformer.layer.0.attention.q_lin.bias', 'distilbert.transformer.layer.0.attention.k_lin.weight', 'distilbert.transformer.layer.0.attention.k_lin.bias', 'distilbert.transformer.layer.0.attention.v_lin.weight', 'distilbert.transformer.layer.0.attention.v_lin.bias', 'distilbert.transformer.layer.0.attention.out_lin.weight', 'distilbert.transformer.layer.0.attention.out_lin.bias', 'distilbert.transformer.layer.0.sa_layer_norm.weight', 'distilbert.transformer.layer.0.sa_layer_norm.bias', 'distilbert.transformer.layer.0.ffn.lin1.weight', 'distilbert.transformer.layer.0.ffn.lin1.bias', 'distilbert.transfor

config.__class__:     <class 'transformers.models.bert.configuration_bert.BertConfig'>
tokenizer.__class__:  <class 'transformers.models.bert.tokenization_bert.BertTokenizer'>
model.__class__:      <class 'transformers.models.bert.modeling_bert.BertModel'>


In [45]:
# 单个句子
# sentence = ["After stealing money from the bank vault", "the bank robber was seen fishing on the Mississippi river bank."]

# 两个句子
sentence = ["After stealing money from the bank vault", "the bank robber was seen fishing on the Mississippi river bank."]

# return_tensors 指定返回的结果为 torch.Tensor，这样就不用做转换了，
# 注意，必须要设置 padding，否则得到的 tensor 维度不一样，无法转成 tensors
sentence_encode = tokenizer(sentence, padding=True, return_tensors='pt')

print(sentence_encode)
print(tokenizer.decode(sentence_encode['input_ids'][0]))
print(tokenizer.decode(sentence_encode['input_ids'][1]))

{'input_ids': tensor([[  101,  2044, 11065,  2769,  2013,  1996,  2924, 11632,   102,     0,
             0,     0,     0,     0],
        [  101,  1996,  2924, 27307,  2001,  2464,  5645,  2006,  1996,  5900,
          2314,  2924,  1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
[CLS] after stealing money from the bank vault [SEP] [PAD] [PAD] [PAD] [PAD] [PAD]
[CLS] the bank robber was seen fishing on the mississippi river bank. [SEP]


In [53]:
# 将加载的预训练模型置于 evaluation 状态，这样会关闭其中的 dropout
model.eval()

# 预测时，使用 .no_grad() 方式会加快计算
with torch.no_grad():
    # 可以直接将 tokenizer 得到的输出作为输入，只要使用拆包技巧就行
    outputs = model(**sentence_encode, output_hidden_states=True, output_attentions=True)
    
outputs.__class__

transformers.modeling_outputs.BaseModelOutputWithPoolingAndCrossAttentions

In [107]:
# 两个句子-batch_size=2, 每个句子序列的长度 sequence_length = 14, 最后一层的 hidden_size = 768
outputs.last_hidden_state.shape

torch.Size([2, 14, 768])

In [114]:
# 对应于 [CLS] token 的 embedding，2 个句子，所以返回了两个
outputs.pooler_output.shape

torch.Size([2, 768])

In [115]:
# 隐状态
print(len(outputs.hidden_states))
outputs.hidden_states.__class__

13


tuple

In [96]:
# Embedding 层的 隐状态
outputs.hidden_states[0].shape

torch.Size([2, 14, 768])

In [109]:
# 第一个 self-attention 层的 隐状态
outputs.hidden_states[1].shape

torch.Size([2, 14, 768])

In [116]:
# attention 的信息
print(len(outputs.attentions))
outputs.attentions.__class__

12


tuple

In [117]:
# 第一个 self-attention 层的 atttention shape
# 2 个句子 batch_size=2, 12 个 heads, sequence_length=14, sequence_length=14
outputs.attentions[0].shape

torch.Size([2, 12, 14, 14])

+ 也可以用 切片 的方式，直接从 `outputs` 中获取前两个的值

In [60]:
# t1 对应于 last_hidden_state, t2 对应于 pooler_output
t1, t2 = outputs[:2]
print(t1.__class__)
print(t1.shape)
print(t2.shape)

<class 'torch.Tensor'>
torch.Size([2, 14, 768])
torch.Size([2, 768])


In [48]:
model.eval()
# return_dict=False，那么返回的就是一个 tuple
outputs = model(**sentence_encode, output_hidden_states=True, output_attentions=True, return_dict=False)

In [50]:
print("outputs.__class__: ", outputs.__class__)
print("outputs.len:       ", len(outputs))

outputs.__class__:  <class 'tuple'>
outputs.len:        4


In [52]:
outputs[0].shape

torch.Size([2, 14, 768])

#### BertForPreTraining

在 BertModel 的输出后，增加了一层 `BertPreTrainingHeads()`，如果要自己训练一个BERT模型，从这个类出发比较方便.

+ 通过分析源码可以发现，它（并行）做了如下两件事：
  1. 调用 `nn.Linear()` 将 BertModel 输出的 `[CLS]` token 映射成 2 维向量，用于表示二分类的值——用于 Next Sentence 的训练方式
  2. 调用 `BertLMPredictionHead()` ，将 BertModel 最后一层输出 last_hidden_state 进行处理，内部处理逻辑如下：
    + 调用 `BertPredictionHeadTransform()` 对 last_hidden_state 做 线性映射 + 激活函数 + LayerNormalization
    + 调用 `nn.Linear()` 将上一步得到的向量映射成 vocab_size 维度的向量，用于做 MaskLanguageModel 的训练


+ `forward()`方法，参数相比于 `BertModel.forward()` 多了如下两个：
  + `label`, shape=`(batch_size, sequence_length)`，用于计算 masked language modeling 的 Loss.   
  其中的取值范围应为 `[-100, 0, ..., config.vocab_size]`，也就是和 `input_ids` 一样，-100 表示对应位置的token 会被 mask.
  + `next_sentence_label`, shape=`(batch_size,)`，用于计算 next sequence prediction(分类问题) 的 Loss.   
  取值只有 `{0, 1}`
    + 0 表示 后一句 是连着 前一句
    + 1 表示 后一句 是随机抽取的


+ 返回值是一个封装的 `BertForPreTrainingOutput()` 对象（`return_dict=True`时）
  + `loss`，shape=`(1,)`当`labels`提供了时，保存了 MaskLM 的 Loss 和 next sentence prediction 的 Loss
  + `prediction_logits`，shape=`(batch_size, sequence_length, config.vocab_size)`，记录了softmax之前的score
  + `seq_relationship_logits`, shape=`(batch_size, 2)`，记录了 next sequence prediction 在 softmax 之前的 score
  + `hidden_states`，就是 `BertModel` 的 hidden_states
  + `attentions`，`BertModel`的对应输出

In [8]:
from transformers import BertTokenizer, BertConfig, BertForPreTraining

In [11]:
model_name = "./BERT/bert-pre-trained-models/bert-base-uncased"
# model_name = "./BERT/bert-pre-trained-models/distilbert-base-uncased/"

config = BertConfig.from_pretrained(model_name, local_files_only=True)
tokenizer = BertTokenizer.from_pretrained(pretrained_model_name_or_path=model_name, local_files_only=True)
model = BertForPreTraining.from_pretrained(model_name, local_files_only=True)

print("config.__class__:    ", config.__class__)
print("tokenizer.__class__: ", tokenizer.__class__)
print("model.__class__:     ", model.__class__)

Some weights of BertForPreTraining were not initialized from the model checkpoint at ./BERT/bert-pre-trained-models/bert-base-uncased and are newly initialized: ['cls.predictions.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


config.__class__:     <class 'transformers.models.bert.configuration_bert.BertConfig'>
tokenizer.__class__:  <class 'transformers.models.bert.tokenization_bert.BertTokenizer'>
model.__class__:      <class 'transformers.models.bert.modeling_bert.BertForPreTraining'>


In [19]:
# 两个句子
sentence = ["After stealing money from the bank vault", "the bank robber was seen fishing on the Mississippi river bank."]

# return_tensors 指定返回的结果为 torch.Tensor，这样就不用做转换了，
# 注意，必须要设置 padding，否则得到的 tensor 维度不一样，无法转成 tensors
sentence_encode = tokenizer(sentence, padding=True, return_tensors='pt')
print(sentence_encode)

outputs = model(**sentence_encode, output_hidden_states=True, output_attentions=True)

{'input_ids': tensor([[  101,  2044, 11065,  2769,  2013,  1996,  2924, 11632,   102,     0,
             0,     0,     0,     0],
        [  101,  1996,  2924, 27307,  2001,  2464,  5645,  2006,  1996,  5900,
          2314,  2924,  1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [13]:
outputs.__class__

transformers.models.bert.modeling_bert.BertForPreTrainingOutput

In [14]:
outputs.loss

In [17]:
outputs.prediction_logits.shape

torch.Size([2, 14, 30522])

In [18]:
outputs.seq_relationship_logits.shape

torch.Size([2, 2])

#### BertForSequenceClassification

这个类专门用于 序列分类，它的内部很简单：
1. `BertModel()` 输出 pooled_output(`[CLS]` token 对应的 hidden_state)
2. `nn.Dropout()` 处理 pooled_output
3. `nn.Linear()` 映射到 多分类的类别 logits


+ `forward()`方法

+ 返回的是`SequenceClassifierOutput()`类对象

---

### Distil-BERT 模型

[DistilBERT Models](https://huggingface.co/transformers/model_doc/distilbert.html) 包含如下类:   